#### Installing EMNIST, only run 1 time

In [ ]:
!pip uninstall emnist


Found existing installation: emnist 0.0
Uninstalling emnist-0.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/emnist-0.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/emnist/*
Proceed (Y/n)? 

In [ ]:
!pip install emnist


#### Loading Data

In [ ]:
import numpy as np
import gzip
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

def load_emnist_images(filename):
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1, 28, 28)
    return data

def load_emnist_labels(filename):
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=8)
    return data

# Google Drive folder path
folder_path = '/content/drive/My Drive/A Final Project/Data'

# Load training images and labels
x_train = load_emnist_images(os.path.join(folder_path, 'emnist-letters-train-images-idx3-ubyte.gz'))
y_train = load_emnist_labels(os.path.join(folder_path, 'emnist-letters-train-labels-idx1-ubyte.gz'))

# Load test images and labels
x_test = load_emnist_images(os.path.join(folder_path, 'emnist-letters-test-images-idx3-ubyte.gz'))
y_test = load_emnist_labels(os.path.join(folder_path, 'emnist-letters-test-labels-idx1-ubyte.gz'))

# Load mapping
with open(os.path.join(folder_path, 'emnist-letters-mapping.txt'), 'r') as f:
    mapping = f.readlines()
mapping = [line.split() for line in mapping]
mapping = {int(line[0]): chr(int(line[1])) for line in mapping}

# Convert labels to characters
y_train = np.array([mapping[label] for label in y_train])
y_test = np.array([mapping[label] for label in y_test])

In [ ]:
import matplotlib.pyplot as plt

# Function to plot images
def plot_images(images, labels, nrows, ncols):
    fig, axes = plt.subplots(nrows, ncols, figsize=(10, 10))
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(28, 28), cmap='gray')
        ax.set_title(labels[i])
        ax.axis('off')

# Plot first 25 images from the training set
plot_images(x_train, y_train, 5, 5)

# Print first 25 labels from the training set
print("First 25 labels from the training set:")
for label in y_train[:25]:
    print(label, end=' ')


#### Preprocessing

In [ ]:
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

from sklearn.preprocessing import OneHotEncoder

# One-hot encode the labels
encoder = OneHotEncoder(sparse=False)
y_train_onehot = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_onehot = encoder.transform(y_test.reshape(-1, 1))

# Verify the shape and unique classes
print(f"Shape of y_train_onehot: {y_train_onehot.shape}")
print(f"Unique classes in y_train: {np.unique(y_train)}")

#### Model 1

In [ ]:
# Model definition and compilation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.utils import set_random_seed
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

# This is needed for replicability
set_random_seed(555)
total_num_classes = 26  # For the 26 alphabet letters

model1 = Sequential()
model1.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1,), activation="relu"))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(32, kernel_size=(2, 2), activation="relu"))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Flatten())
model1.add(Dense(total_num_classes, activation='softmax'))

# Configure optimizer and loss
optim = Adam(learning_rate=0.001)
loss_fn = CategoricalCrossentropy()

model1.compile(loss=loss_fn, optimizer=optim, metrics=["accuracy"])

batch_size = 128
epochs = 15

model1.fit(x_train, y_train_onehot, batch_size=batch_size, epochs=epochs)


In [ ]:
from sklearn.metrics import accuracy_score

# Create a mapping dictionary
mapping = {i: chr(i + 65) for i in range(26)}

predictions1 = model1.predict(x_test)
predicted_class_indices1 = predictions1.argmax(axis=1)
true_class_indices = np.argmax(y_test_onehot, axis=1)

# Get the class labels for predictions and true labels
predicted_class_labels1 = [mapping[i] for i in predicted_class_indices1]
true_class_labels = [mapping[i] for i in true_class_indices]

# Compute accuracy
acc1 = accuracy_score(true_class_labels, predicted_class_labels1)

print(f"Testing accuracy: Model 1 = {acc1}")


In [ ]:
model1.summary()

In [ ]:
from sklearn.metrics import classification_report

report1 = classification_report(true_class_labels, predicted_class_labels1)
print("Classification Report: Model 1\n", report1)

#### Model 2: Changing Activation Function

In [ ]:
model2 = Sequential()
model2.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1,), activation="sigmoid"))  # Changed activation function to sigmoid
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(32, kernel_size=(2, 2), activation="sigmoid"))  # Changed activation function to sigmoid
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Dense(total_num_classes, activation='softmax'))

from tensorflow.keras.optimizers import legacy

optim = legacy.Adam(learning_rate=0.001)

model2.compile(loss=loss_fn, optimizer=optim, metrics=["accuracy"])

batch_size = 128
epochs = 15

model2.fit(x_train, y_train_onehot, batch_size=batch_size, epochs=epochs)

In [ ]:
model2.summary()

In [ ]:
from sklearn.metrics import accuracy_score

# Create a mapping dictionary
mapping = {i: chr(i + 65) for i in range(26)}

predictions2 = model2.predict(x_test)
predicted_class_indices2 = predictions2.argmax(axis=1)
true_class_indices = np.argmax(y_test_onehot, axis=1)

# Get the class labels for predictions and true labels
predicted_class_labels2 = [mapping[i] for i in predicted_class_indices2]
true_class_labels = [mapping[i] for i in true_class_indices]

# Compute accuracy
acc2 = accuracy_score(true_class_labels, predicted_class_labels2)

print(f"Testing accuracy: Model 2 = {acc2}")

In [ ]:
report2 = classification_report(true_class_labels, predicted_class_labels2)
print("Classification Report: Model 2\n", report2)

#### Model 3: More Layers

In [ ]:
model3 = Sequential()
model3.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation="sigmoid"))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Conv2D(64, kernel_size=(2, 2), activation="sigmoid"))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Conv2D(128, kernel_size=(2, 2), activation="sigmoid"))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Flatten())
model3.add(Dense(128, activation="sigmoid"))
model3.add(Dense(total_num_classes, activation="softmax"))

model3.compile(loss=loss_fn, optimizer=optim, metrics=["accuracy"])

batch_size = 128
epochs = 15

model3.fit(x_train, y_train_onehot, batch_size=batch_size, epochs=epochs)


In [ ]:
model3.summary()

In [ ]:
from sklearn.metrics import accuracy_score

# Create a mapping dictionary
mapping = {i: chr(i + 65) for i in range(26)}

predictions3 = model3.predict(x_test)
predicted_class_indices3 = predictions3.argmax(axis=1)
true_class_indices = np.argmax(y_test_onehot, axis=1)

# Get the class labels for predictions and true labels
predicted_class_labels3 = [mapping[i] for i in predicted_class_indices3]
true_class_labels = [mapping[i] for i in true_class_indices]

# Compute accuracy
acc3 = accuracy_score(true_class_labels, predicted_class_labels3)

print(f"Testing accuracy: Model 3 = {acc3}")

In [ ]:
report3 = classification_report(true_class_labels, predicted_class_labels3)
print("Classification Report: Model 3\n", report3)

#### Model 4: Increased Filters

In [ ]:
model4 = Sequential()
model4.add(Conv2D(64, kernel_size=(3, 3), input_shape=(28, 28, 1), activation="sigmoid"))  # Increased number of filters
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Conv2D(128, kernel_size=(2, 2), activation="sigmoid"))  # Increased number of filters
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Conv2D(256, kernel_size=(2, 2), activation="sigmoid"))  # Increased number of filters
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Flatten())
model4.add(Dense(128, activation="sigmoid"))
model4.add(Dense(total_num_classes, activation="softmax"))

model4.compile(loss=loss_fn, optimizer=optim, metrics=["accuracy"])

batch_size = 128
epochs = 15

model4.fit(x_train, y_train_onehot, batch_size=batch_size, epochs=epochs)

In [ ]:
model4.summary()

In [ ]:
# Create a mapping dictionary
mapping = {i: chr(i + 65) for i in range(26)}

predictions4 = model4.predict(x_test)
predicted_class_indices4 = predictions4.argmax(axis=1)
true_class_indices = np.argmax(y_test_onehot, axis=1)

# Get the class labels for predictions and true labels
predicted_class_labels4 = [mapping[i] for i in predicted_class_indices4]
true_class_labels = [mapping[i] for i in true_class_indices]

# Compute accuracy
acc4 = accuracy_score(true_class_labels, predicted_class_labels4)

print(f"Testing accuracy: Model 4 = {acc4}")

In [ ]:
report4 = classification_report(true_class_labels, predicted_class_labels3)
print("Classification Report: Model 4\n", report4)

#### Additional evaluation with visualizations

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
import numpy as np

# Binarize the true labels
y_test_binarized = label_binarize(y_test, classes=list(mapping.values()))

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(len(mapping)):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], predictions4[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_binarized.ravel(), predictions4.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot ROC curve for each class
plt.figure(figsize=(10, 8))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange', 'purple', 'brown']

for i, color in zip(range(len(mapping)), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(mapping[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Compute confusion matrix
conf_matrix = confusion_matrix(true_class_labels, predicted_class_labels4)

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(mapping.values()),
            yticklabels=list(mapping.values()))
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
print("Unique values in predicted_class_indices:", np.unique(predicted_class_indices3))
print("Unique values in true_class_indices:", np.unique(true_class_indices))


In [ ]:
missing_keys = [i for i in predicted_class_indices3 if i not in mapping.keys()]
print("Missing keys:", missing_keys)


In [ ]:
plt.imshow(x_test[0],cmap="gray")
plt.title(f"Predicted = {y_test[0]} Expected = {predicted_class_indices3[0]}")

In [ ]:
plt.imshow(x_test[1],cmap="gray")
plt.title(f"Predicted = {y_test[1]} Expected = {predicted_class_indices3[1]}")

In [ ]:
plt.imshow(x_test[2],cmap="gray")
plt.title(f"Predicted = {y_test[2]} Expected = {predicted_class_indices3[2]}")

# Code below is old!

In [ ]:
from sklearn.preprocessing import OneHotEncoder

y_train = [[y] for y in y_train]

encoder = OneHotEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train).toarray()

print(y_train)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(x_train[0],cmap="gray")
plt.title(f"Training Example with Label = {str(y_train[0])}")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense
from tensorflow.keras.utils import set_random_seed

# This is needed for replicability
set_random_seed(555)
total_num_classes = 2

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1,), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(2, 2), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(total_num_classes, activation='sigmoid'))

# visualize the model design
model.summary()

In [ ]:
unique_classes = np.unique(y_train)
num_unique_classes = len(unique_classes)
print(f"Number of unique classes: {num_unique_classes}")
print(f"Unique classes: {unique_classes}")


In [ ]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

optim = Adam(learning_rate=0.001)
loss_fn = BinaryCrossentropy()

model.compile(loss=loss_fn, optimizer=optim, metrics=["accuracy"])

batch_size = 128
epochs = 15

# Ensure y_train is reshaped for binary classification
y_train_binary = y_train.reshape(-1, 1)

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)